<a href="https://colab.research.google.com/github/shaonsani/Sentiment-Analysis-using-LSTM-and-Kaggle-Dataset/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
files.upload()

In [ ]:
data = pd.read_csv('Tweets.csv',sep=',')
data.head(2)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
tweet_df = data[['text','airline_sentiment']]
tweet_df = tweet_df[tweet_df['airline_sentiment']!='neutral']

In [ ]:
sentiment_label = tweet_df.airline_sentiment.factorize()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tweet = tweet_df.text.values 
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)

vocab_size = len(tokenizer.word_index)+1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequences = pad_sequences(encoded_docs,maxlen=200)


In [ ]:
print(encoded_docs[0])
print(padded_sequences[77])

[2, 340, 400, 500, 987, 1, 4, 112, 988]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    2  189  122  721
 1151   52    6  670   18    6  722 1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout, SpatialDropout1D, Embedding

embedding_vector_lenght=32
model =Sequential()
model.add(Embedding(vocab_size, embedding_vector_lenght, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(100,dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           76608     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 200, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 129,909
Trainable params: 129,909
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(padded_sequences,sentiment_label[0],validation_split=0.2,epochs=20, batch_size=32)


Epoch 1/20
16/16 [==============================] - 12s 722ms/step - loss: 0.6726 - accuracy: 0.6267 - val_loss: 0.4261 - val_accuracy: 0.8492
Epoch 2/20
16/16 [==============================] - 11s 700ms/step - loss: 0.6329 - accuracy: 0.6647 - val_loss: 0.5466 - val_accuracy: 0.8651
Epoch 3/20
16/16 [==============================] - 11s 696ms/step - loss: 0.5684 - accuracy: 0.7265 - val_loss: 0.5475 - val_accuracy: 0.7460
Epoch 4/20
16/16 [==============================] - 11s 693ms/step - loss: 0.4857 - accuracy: 0.7685 - val_loss: 0.4450 - val_accuracy: 0.8016
Epoch 5/20
16/16 [==============================] - 12s 719ms/step - loss: 0.3791 - accuracy: 0.8363 - val_loss: 0.4642 - val_accuracy: 0.8492
Epoch 6/20
16/16 [==============================] - 11s 686ms/step - loss: 0.3275 - accuracy: 0.8623 - val_loss: 0.4792 - val_accuracy: 0.8333
Epoch 7/20
16/16 [==============================] - 11s 692ms/step - loss: 0.2518 - accuracy: 0.9142 - val_loss: 0.4469 - val_accuracy: 0.7937

In [ ]:
test_text = 'the product is fabulous'
tw = tokenizer.texts_to_sequences([test_text])
pw = pad_sequences(tw, maxlen=200)
prediction = int(model.predict(pw).round())
sentiment_label[1][prediction]


'positive'